# Проект команды 306 - Проектный семинар - №2 (Выгрузка картинок)

Состав команды:

1. Алиев Хайрутдин Аллилович
2. Зубов Дмитрий Сергеевич
3. Курбанов Иван Сергеевич
4. Лухнев Игорь Дмитриевич
5. Шишков Максим Алексеевич

## TL;DR
---
Данный документ нужен для создания базы с изображениями с сайта FF, именно их мы будем использовать для генерации признаков

---

## Предварительные зависимости
В следующей ячейке будут установлены необходимые библиотеки через `pip`

In [1]:
# Use !pip install -q [your-package]
!pip install -q PyGithub
!pip install -q fsspec

Следующая ячейка для импорта установленных пакетов в проект

In [2]:
# use aliases for long names
import os
import requests
from github import Github as gh
from getpass import getpass
import re
import pandas as pd
import numpy as np
import base64
from io import StringIO
from pathlib import Path

## Загрузка данных 

Запросим у пользователя креды для подключения к репозиторию

In [3]:
login = input('Enter your login: ')
password = getpass('Enter the secret value: ')

Подключимся к репозиторию

In [4]:
repo_path = "IgorLukhnev/FarFetchRS"
# Запишем токен в переменную окружения
g_token = os.getenv('GITHUB_TOKEN', password)
# Вополним коннект
g = gh(g_token)
# Подключимся к репо
repo = g.get_repo(repo_path)

Данные будем загружать на основе датасета, подготовленного на этапе EDA

In [5]:
dir_path = 'Data/simpleCat'
data_path = 'Data/simpleCat/final_data.csv'
contents = repo.get_contents(dir_path)
for x in contents:
    if x.path == data_path:
        data = pd.read_csv(StringIO(base64.b64decode(repo.get_git_blob(x.sha).content).decode("utf8"))).drop(columns=['Unnamed: 0'])

data.head()

## Загружаем картинки по папкам

Создадим для каждой строки путь папки, в которую картинка будет записана

In [6]:
data['folder'] = data['sex'] + '_' + data['cat'] + '_' + data['category1'] + ',' + data['img'] + ',' + data.index.astype(str)
data.head()

Напишем функцию для обработки датафрейма и загрузки фото

In [7]:
def load_image(cmnd_string):
    # Парсим строку на имя папки, ссылку на фото, имя фото
    folder, url, img_name = cmnd_string.split(',')
    # получаем в битах картинку
    img_data = requests.get(url).content
    # формируем путь для сохранения
    path = folder + '/' + img_name + '.jpg'
    # создаем нужную папку
    Path(folder + '/').mkdir(parents=True, exist_ok=True)
    with open(path, 'wb') as handler:
        handler.write(img_data)

Попробуем применить то, что написали тут

In [ ]:
data['folder'].apply(lambda x: load_image(x))

In [ ]:
!zip -r ./ data